In [ ]:
!pip install transformers
!pip install torch
!pip install torchtext==0.6.0
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
# Libraries
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torchtext.data import Field,TabularDataset,BucketIterator,Iterator
from sklearn.metrics import classification_report, confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm.notebook import tqdm_notebook as tqdm
import gc
gc.collect()
torch.cuda.empty_cache()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#  Check for CUDA to run  code in GPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Download Model

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
MAX_SEQ_LENGTH = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

In [ ]:
df= pd.read_csv('/content/train3.csv')
df

,label,review
0,1,insanity but the millennial fools will suck it...
1,0,expose them racists homophobes islamophobes et...
2,1,niggers gonna nig
3,2,<number> what the only place in the world wher...
4,0,the mcimpeachment mcfiasco is all about angeri...
...,...,...
8201,1,no sand niggers no terror pretty straightforward
8202,0,likewise my pinned tweet on inbreeding in the ...
8203,2,that feel when the women in our movement are a...
8204,2,sml and edi wow both radiate dumb bitch energy...


In [ ]:
# Fields
label_field = Field(sequential=False,use_vocab=False,batch_first=False,dtype=torch.float)
text_field = Field(use_vocab=False,tokenize=tokenizer.encode,lower=False,include_lengths=False,batch_first=True,fix_length=MAX_SEQ_LENGTH,pad_token=PAD_INDEX,unk_token=UNK_INDEX)
fields = [('label',label_field),('review',text_field)]


In [ ]:
# Tabular Dataset
train, valid, test= TabularDataset.splits( path="/content/", train='train3.csv', validation='valid3.csv',
                                             test='test3.csv',format='CSV', fields=fields, skip_header=True)
# Iterators
train_iter = BucketIterator(train, batch_size=16, sort_key=lambda x: len(x.review),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=16, sort_key=lambda x: len(x.review),
                            device=device, train=True, sort=True, sort_within_batch=True)
test_iter = Iterator(test, batch_size=16, device=device, train=False, shuffle=False, sort=False)


### Model 

In [ ]:
class ViolationBert(nn.Module):

    def __init__(self):
        super(ViolationBert,self).__init__()
        self.encoder = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
    #The forward method is used for the actual forward pass through the model. It takes in two inputs, text and label, 
    # and passes text through the encoder to obtain loss and logits (the unnormalized prediction scores) for the given input text and label pair. 
    #The loss and logits are then returned as a tuple.
    def forward(self, text, label):
      loss, logits = self.encoder(text, labels=label)[:2]
      return loss, logits
        

In [ ]:
#two functions save_checkpoint and load_checkpoint which can be used to save and load PyTorch model checkpoints.

def save_checkpoint(save_path,model,valid_loss):

    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}

    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path,model):

    state_dict = torch.load(load_path,map_location= device)
    print(f'Model loaded from <== {load_path}')

    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

In [ ]:
# PyTorch function that is used to free up memory in the CUDA memory allocator. 
torch.cuda.empty_cache()

In [ ]:
# Defines two functions save_metrics and load_metrics which can be used to 
# save and load the training and validation loss, as well as the global steps at which the losses were computed, for a PyTorch model.
def save_metrics(save_path,train_loss_list,valid_loss_list,global_steps_list):

    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}

    torch.save(state_dict, save_path)
    print(f'Model metrics saved to ==> {save_path}')

def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']


In [ ]:
### Model Training

def training(model, 
             optimizer,
             train_loader, 
             valid_loader ,
             eval_every ,
             num_epochs = 5,
             criterion = nn.CrossEntropyLoss(),
             file_path = '',
             best_valid_loss = float("Inf")):

    running_loss = 0.0
    validation_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []  

    model.train()
    for epoch in tqdm(range(num_epochs)):
       for (label, review), _ in train_loader:
            label= label.type(torch.LongTensor).to(device)
            review = review.type(torch.LongTensor).to(device)

            loss,_ = model(review,label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():

                    for (label, review), _ in valid_loader:

                        label = label.type(torch.LongTensor).to(device)
                        review = review.type(torch.LongTensor).to(device)

                        loss,_ = model(review,label)

                        validation_running_loss += loss.item()
            
                average_train_loss = running_loss / eval_every
                average_valid_loss = validation_running_loss / len(valid_loader)

                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)


                # resetting running values
                running_loss = 0.0                
                validation_running_loss = 0.0
                model.train()

                 # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))
                
                 # checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path + '/' + 'ba_model.pt', model, best_valid_loss)
                    save_metrics(file_path + '/' + 'ba_metrics.pt', train_loss_list, valid_loss_list, global_steps_list)


    print('Finished Training!')

In [ ]:
# This code creates an instance of the ViolationBert class and assigns it to the classification_model variable. 
# ViolationBert is a PyTorch nn.Module that represents a BERT-based model for sequence classification, which has been pretrained on hate speech data
classification_model = ViolationBert().to(device)
optimizer = torch.optim.Adam(classification_model.parameters(),lr = 2e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
training(model= classification_model,optimizer=optimizer, train_loader = train_iter, valid_loader = valid_iter, eval_every = len(train_iter) // 2)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch [1/5], Step [504/5040], Train Loss: 0.9450, Valid Loss: 0.8742
Model saved to ==> /ba_model.pt
Model metrics saved to ==> /ba_metrics.pt
Epoch [1/5], Step [1008/5040], Train Loss: 0.8940, Valid Loss: 0.8343
Model saved to ==> /ba_model.pt
Model metrics saved to ==> /ba_metrics.pt
Epoch [2/5], Step [1512/5040], Train Loss: 0.7892, Valid Loss: 0.8679
Epoch [2/5], Step [2016/5040], Train Loss: 0.7898, Valid Loss: 0.8447
Epoch [3/5], Step [2520/5040], Train Loss: 0.6807, Valid Loss: 0.9415
Epoch [3/5], Step [3024/5040], Train Loss: 0.6340, Valid Loss: 0.9220
Epoch [4/5], Step [3528/5040], Train Loss: 0.5015, Valid Loss: 1.2174
Epoch [4/5], Step [4032/5040], Train Loss: 0.4529, Valid Loss: 1.2282
Epoch [5/5], Step [4536/5040], Train Loss: 0.3673, Valid Loss: 1.2829
Epoch [5/5], Step [5040/5040], Train Loss: 0.3351, Valid Loss: 1.2220
Finished Training!


### Evaluation

In [ ]:

# model evaluation, its takes test data and calculate accuracy, F1 scores
def evaluate(model,test_loader):
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
        for (label,review),_ in test_loader:  
            label = label.type(torch.LongTensor).to(device)
            review = review.type(torch.LongTensor).to(device)
            _,output = model(review,label)
            y_pred.extend(torch.argmax(output,1).tolist())
            y_true.extend(label.tolist())
    print(classification_report(y_true, y_pred, labels=[1,0,2], digits=4))

In [ ]:
# Load Model from check point and Evaluate

best_model = ViolationBert().to(device)
load_checkpoint('/ba_model.pt',best_model) # your checkpoint location 
evaluate(best_model, test_iter)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Model loaded from <== /ba_model.pt
              precision    recall  f1-score   support

           1     0.7061    0.7479    0.7264       607
           0     0.6016    0.8429    0.7021       815
           2     0.5907    0.2197    0.3203       578

    accuracy                         0.6340      2000
   macro avg     0.6328    0.6035    0.5829      2000
weighted avg     0.6301    0.6340    0.5991      2000



In [ ]:
# check input sentences
def predict_sentiment(model, sentence):
    model.eval()
    tokens = tokenizer.encode_plus(sentence, max_length=MAX_SEQ_LENGTH, truncation=True, padding='max_length', add_special_tokens=True, return_token_type_ids=False, return_attention_mask=True, return_tensors='pt').to(device)
    labels = torch.tensor(0).unsqueeze(0).to(device)
    outputs = model(tokens['input_ids'], labels)
    _, predicted = torch.max(outputs[1], 1)
    return predicted.item()
    

In [ ]:
sentence = "Input your test sentence"
predicted_class = predict_sentiment(best_model, sentence)
print(predicted_class)